In [1]:
pip install z3-solver

Note: you may need to restart the kernel to use updated packages.


In [2]:
from z3 import *

# Propositional Logic

## Declare Variables

In [3]:
p1 = Bool("p1") # (declare-const p1 Bool)
p2 = Bool("p2") # (declare-const p2 Bool)
p3 = Bool("p3") # (declare-const p3 Bool)
p4 = Bool("p4") # (declare-const p4 Bool)

In [4]:
Bool("a b c")

a b c

## Connectives

In [5]:
And(p1, p2) # (and p1 p2)

And(p1, p2)

In [6]:
And(p1,p2,p3,p4, True) # (and p1 p2 p3 p4 true)

And(p1, p2, p3, p4, True)

In [7]:
And(p1,p2,p3,p4, False) # (and p1 p2 p3 p4 false)

And(p1, p2, p3, p4, False)

In [8]:
Or(p1,p2) # (or p1 p2)

Or(p1, p2)

In [9]:
Implies(p1,p2) # (=> p1 p2)

Implies(p1, p2)

In [10]:
If(p1,p2,p3) # (ite p1 p2 p3)

If(p1, p2, p3)

In [11]:
p1==p2 # (= p1 p2)

p1 == p2

## Simplify

In [12]:
And(Or(p1,Not(p2)),Or(p1, Not(p2))) # (and (or p1 (not p2)) (or (not p1) (not p2)))

And(Or(p1, Not(p2)), Or(p1, Not(p2)))

In [13]:
simplify(And(Or(p1,Not(p2)), Or(p1, Not(p2)))) # (simplify (and (or p1 (not p2)) (or (not p1) (not p2))))

Or(p1, Not(p2))

In [14]:
simplify(And(p1,(Or(p1,p2)))) # (simplify (and p1 (or p1 p2)))

And(p1, Or(p1, p2))

In [15]:
simplify(If(True, p2, p3)) # (ite true p2 p3)

p2

## Solving

In [16]:
And(Or(p1,p2), Or(Not(p1),Not(p2))) # (and (or p1 p2) (or (not p1) (not p2)))

And(Or(p1, p2), Or(Not(p1), Not(p2)))

In [17]:
solve(And(Or(p1,p2), Or(Not(p1),Not(p2)))) 

[p1 = True, p2 = False]


In [18]:
And(Or(Not(p1),p2), Or(p1,Not(p2))) # (and (or (not p1) p2) (or p1 (not p2)))

And(Or(Not(p1), p2), Or(p1, Not(p2)))

In [19]:
solve(And(Or(Not(p1),p2), Or(p1,Not(p2))))

[p2 = False, p1 = False]


### Alternative

In [20]:
solver1 = Solver() 
solver1.add(And(Or(Not(p1),p2), Or(p1,Not(p2)))) # (assert (and (or p1 p2) (or (not p1) (not p2))))
solver1.check()  # (check-sat)

sat

In [21]:
solver1.add(And(Or(p1,p2), Or(Not(p1),Not(p2)))) # (assert (and (or (not p1) p2) (or p1 (not p2))))
solver1.check() # (check-sat)

unsat

It checks for the conjunct of all the asserted statements.

In [22]:
solver1

[And(Or(Not(p1), p2), Or(p1, Not(p2))),
 And(Or(p1, p2), Or(Not(p1), Not(p2)))]

----
*Fix 1:* Choose different variables

In [23]:
solver1 = Solver() 
solver1.add(And(Or(Not(p1),p2), Or(p1,Not(p2)))) # (assert (and (or p1 p2) (or (not p1) (not p2))))
solver1.check() # (check-sat)

sat

In [24]:
solver1.add(And(Or(p3,p4), Or(Not(p3),Not(p4)))) # (assert (and (or (not p1) p2) (or p1 (not p2))))
solver1.check() # (check-sat)

sat

In [ ]:
solver1

----
*Fix 2:* Reset solver
-- Warning resets everything!!

In [25]:
solver1 = Solver() 
solver1.add(And(Or(Not(p1),p2), Or(p1,Not(p2)))) # (assert (and (or p1 p2) (or (not p1) (not p2))))
solver1.check() # (check-sat)

sat

In [26]:
solver1

[And(Or(Not(p1), p2), Or(p1, Not(p2)))]

In [27]:
solver1.reset() # (reset)
solver1

[]

In [28]:
solver1.add(And(Or(p1,p2), Or(Not(p1),Not(p2)))) # (assert (and (or (not p1) p2) (or p1 (not p2))))
solver1.check() # (check-sat)

sat

In [29]:
solver1

[And(Or(p1, p2), Or(Not(p1), Not(p2)))]

----
*Fix 3:*  Scopes

In [30]:
solver1 = Solver() 
solver1.add(And(Or(Not(p1),p2), Or(p1,Not(p2)))) # (assert (and (or p1 p2) (or (not p1) (not p2))))
solver1

[And(Or(Not(p1), p2), Or(p1, Not(p2)))]

In [31]:
solver1.push() # (push)
solver1.add(And(Or(p1,p2), Or(Not(p1),Not(p2)))) # (assert (and (or (not p1) p2) (or p1 (not p2))))
solver1

[And(Or(Not(p1), p2), Or(p1, Not(p2))),
 And(Or(p1, p2), Or(Not(p1), Not(p2)))]

In [32]:
solver1.check() # (check-sat)

unsat

In [33]:
solver1.pop() # (pop)
solver1

[And(Or(Not(p1), p2), Or(p1, Not(p2)))]

In [34]:
solver1.check() # (check-sat)

sat

In [35]:
solver1.pop()

Z3Exception: b'index out of bounds'

In [ ]:
solver1.push()

In [ ]:
solver1.pop()

## Models

### List all models

In [36]:
def get_solutions(s):
    result = s.check()
    # While we still get solutions
    while (result == sat):
        m = s.model()
        yield m
        # Add new solution as a constraint
        block = []
        for var in m:
            print(var() != m[var])
            block.append(var() != m[var])
        print("add: ", Or(block))
        print()
        s.add(Or(block))
        # Look for new solution
        result = s.check()

In [37]:
solver1 = Solver() 
solver1.add(And(Or(Not(p1),p2), Or(p1,Not(p2)))) # (assert (and (or p1 p2) (or (not p1) (not p2))))
for sol in get_solutions(solver1):
    print("Output: ",sol)

Output:  [p2 = False, p1 = False]
p2 != False
p1 != False
add:  Or(p2 != False, p1 != False)

Output:  [p2 = True, p1 = True]
p2 != True
p1 != True
add:  Or(p2 != True, p1 != True)



### Restrict Models

#### At Least
At least X predicates have to be satisfied.

In [ ]:
Or(And(p1, Not(p1)), p2, And(p3, Not(p3)), p4) # (or (and p1 (not p1)) p2 (and p3 (not p3)) p4)

In [38]:
And(p1, Not(p1)), p2, And(p3, Not(p3)), p4 # (or (and p1 (not p1)) p2 (and p3 (not p3)) p4)

(And(p1, Not(p1)), p2, And(p3, Not(p3)), p4)

In [39]:
solver1 = Solver() 
solver1.add(AtLeast( And(p1, Not(p1)), p2, And(p3, Not(p3)), p4,1  )) 
# (assert ((_ at-least 1) (and p1 (not p1)) p2 (and p3 (not p3)) p4))
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p2 = True, p4 = False]

In [40]:
solver1 = Solver() 
solver1.add(AtLeast( And(p1, Not(p1)), p2, And(p3, Not(p3)), p4 , 2 )) 
# (assert ((_ at-least 2) (and p1 (not p1)) p2 (and p3 (not p3)) p4))
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p2 = True, p4 = True]

In [41]:
solver1 = Solver() 
solver1.add(AtLeast(And(p1, Not(p1)), p2, And(p3, Not(p3)), p4, 3 )) 
# (assert ((_ at-least 3) (and p1 (not p1)) p2 (and p3 (not p3)) p4))
solver1.check() # (check-sat)

unsat

#### At Most
At most X predicates have to be satisfied.

In [42]:
Or(Or(p1, Not(p1)), p2, Or(p3, Not(p3)), p4)

Or(Or(p1, Not(p1)), p2, Or(p3, Not(p3)), p4)

In [43]:
solver1 = Solver() 
solver1.add(AtMost(Or(p1, Not(p1)), p2, Or(p3, Not(p3)), p4, 1 )) 
# (assert ((_ at-most 1) (or p1 (not p1)) p2 (or p3 (not p3)) p4))
solver1.check() # (check-sat)

unsat

In [44]:
solver1 = Solver() 
solver1.add(AtMost(Or(p1, Not(p1)), p2, Or(p3, Not(p3)), p4, 2 )) 
# (assert ((_ at-most 2) (or p1 (not p1)) p2 (or p3 (not p3)) p4))
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p2 = False, p4 = False]

In [45]:
solver1 = Solver() 
solver1.add(AtMost(Or(p1, Not(p1)), p2, Or(p3, Not(p3)), p4, 3 )) 
# (assert ((_ at-most 3) (or p1 (not p1)) p2 (or p3 (not p3)) p4))
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p2 = False, p4 = False]

In [46]:
Or(p1, p2, p3, p4) # (or p1 p2 p3 p4)

Or(p1, p2, p3, p4)

#### Quantitative Restriction
The sum of satisfied predicates should equal some number, e.g., 
    
 $5*p_1+ 4*p_2 + 3*p_3 + 2*p_4 = 6$

In [49]:
solver1 = Solver() 
solver1.add(PbGe(((p1, 5), (p2, 4), (p3, 3), (p4, 2)),6))  # (assert ((_ pbge 6 5 4 3 2) p1 p2 p3 p4))
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p3 = True, p1 = True, p2 = False, p4 = False]

In [50]:
solver1 = Solver() 
solver1.add(PbGe(((p1, 5), (p2, 4), (p3, 3), (p4, 2)),5))  # (assert ((_ pbge 5 5 4 3 2) p1 p2 p3 p4))
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p1 = True]

In [51]:
solver1 = Solver() 
solver1.add(PbGe(((p1, 5), (p2, 4), (p3, 3), (p4, 2)),15))  # (assert ((_ pbge 5 5 4 3 2) p1 p2 p3 p4))
solver1.check() # (check-sat)

unsat

In [52]:
solver1 = Solver() 
solver1.add(PbLe(((p1, 5), (p2, 4), (p3, 3), (p4, 2)),5))  # (assert ((_ pble 1 5 4 3 2) p1 p2 p3 p4))
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p1 = False, p2 = True, p3 = False, p4 = False]

In [53]:
solver1 = Solver() 
solver1.add(PbLe(((p1, 5), (p2, 4), (p3, 3), (p4, 2)),1))  # (assert ((_ pble 1 5 4 3 2) p1 p2 p3 p4))
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p3 = False, p1 = False, p2 = False, p4 = False]

In [54]:
solver1 = Solver() 
solver1.add(PbEq(((p1, 5), (p2, 4), (p3, 3), (p4, 2)),13))  # (assert ((_ pbeq 1 5 4 3 2) p1 p2 p3 p4))
solver1.check() # (check-sat)

unsat

In [55]:
solver1 = Solver() 
solver1.add(PbEq(((p1, 5), (p2, 4), (p3, 3), (p4, 2)), 10))  # (assert ((_ pbeq 1 5 4 3 2) p1 p2 p3 p4))
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p3 = True, p1 = True, p2 = False, p4 = True]

## Declare Alias

In [57]:
e1 = And(Or(p1,p2), Or(Not(p1),Not(p2))) # (define-fun e1 () Bool (and (or p1 p2) (or (not p1) (not p2))))

In [58]:
e1

And(Or(p1, p2), Or(Not(p1), Not(p2)))

In [59]:
solver1 = Solver() 
solver1.add(e1)  # (assert e1)
solver1.check() # (check-sat)
solver1.model() # (get-model)

[p1 = True, p2 = False]

## Integers and More

In [60]:
x = Int('x') # (declare-const x Int)
y = Int('y') # (declare-const y Int)
z = Real('z') # (declare-const z Real)

In [61]:
x > 2, y < 10, x + 2*y == 7 # (and (> x 2) (< y 10) (= 7 (+ x (* 2 y))))

(x > 2, y < 10, x + 2*y == 7)

In [62]:
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


In [63]:
solve(x > 2, y < 10, z<Q(2,3), z > -Q(1,3), z+ x + 2*y == 7)

[x = 7, y = 0, z = 0]


In [64]:
solve(x > 2, y < 10, z<Q(2,3), z > Q(1,3), z+ x + 2*y == 7)

no solution


In [65]:
solve(x > 2, y < 10, z<Q(2,3), z > -Q(1,3), z+ x + 2*y == 7.5)

[x = 7, y = 0, z = 1/2]


In [66]:
simplify(x + y + 2*x + 3)

3 + 3*x + y

In [67]:
And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)

And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)

In [68]:
simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5))

And(x >= 2, 2*x**2 + y**2 >= 3)

## Other Sorts

In [69]:
b = BoolSort() 
s = StringSort() 
a = ArraySort(IntSort(), IntSort()) # (declare-const a Array Int Int)
n = IntSort() # (declare-const n Int)
x = RealSort() # (declare-const x Real)

In [70]:
print(Datatype(b))
print(Datatype(s))
print(Datatype(a))
print(Datatype(n))
print(Datatype(x))

Datatype(Bool, [])
Datatype(String, [])
Datatype(Array(Int, Int), [])
Datatype(Int, [])
Datatype(Real, [])


In [71]:
MySort = DeclareSort("MySort") # (declare-sort MySort)

In [72]:
ArraySort(StringSort(), StringSort())

Array(String, String)

In [73]:
bv1= Const("bv1", b)

In [74]:
bv2 = Bool("bv2")

In [75]:
bv2.sort()

Bool

In [76]:

ms= Const("ms", MySort)
print(ms.sort())
print(Datatype(ms))

MySort
Datatype(ms, [])


In [77]:
TupleSort("pair", [IntSort(), BoolSort(),StringSort()])

(pair, pair, [project0, project1, project2])

In [78]:
pair, pair_constructor, (first, second, third) = TupleSort("pair", [IntSort(), BoolSort(),StringSort()])

In [79]:
p = Const('p', pair)

In [80]:
print(p.sort())
print(Datatype(p))

pair
Datatype(p, [])


In [81]:
x,y,z = Ints("x y z")

## Uninterpreted Functions

In [85]:
x = Int('x') # (declare-const x Int)
y = Int('y') # (declare-const y Int)
f = Function('f', IntSort(),  IntSort()) # (declare-fun f Int Int)
solve(f(f(x)) == x, f(x) == y, x != y) # (and (= (f (f x)) x) (= (f x) y) (not (= x y)))

[x = 0, y = 1, f = [1 -> 0, else -> 1]]


In [84]:
f = Function('f', IntSort(), BoolSort(),  IntSort()) # (declare-fun f Int Int)

## Quantifiers

In [86]:
A = DeclareSort("A")  # (declare-sort A)
x = Const("x",A) # (declare-const x A)
y = Const("y",A) # (declare-const y A)
z = Const("z",A) # (declare-const z A)
P = Function("P", A, A, BoolSort())   # (declare-fun P (A,A) Bool)

In [87]:
reflex = ForAll([x], P(x,x)) # (forall ((x A)) (P x x))
reflex

ForAll(x, P(x, x))

In [88]:
antisym = ForAll([x,y], Implies(And(P(x,y), P(y,x)), x==y))  
# (forall ((x A) (y A)) (=> (and (P x y) (P y x)) (= x y)))
antisym

ForAll([x, y], Implies(And(P(x, y), P(y, x)), x == y))

In [89]:
trans = ForAll([x,y,z], Implies(And(P(x,y), P(y,z)), P(x,z)))
# (forall ((x A) (y A) (z A)) (=> (and (P x y) (P y z)) (P x z)))
trans

ForAll([x, y, z], Implies(And(P(x, y), P(y, z)), P(x, z)))

In [90]:
solver1 = Solver()
solver1.add(reflex, antisym, trans)
solver1.check()
solver1.model()

[P = [else -> True]]

##### Increase domain

In [91]:
restr_domain = Exists([x,y], x!=y)
# (exists ((x A) (y A)) (not (= x y)))

In [92]:
solver1 = Solver()
solver1.add(reflex, antisym, trans, restr_domain)
solver1.check()
solver1.model()

[P = [else ->
      Not(And(Not(Var(0) == A!val!0), Var(1) == A!val!0))]]

##### Alternative

In [93]:
Domain = DeclareSort('Domain')
a, b, c = Consts('a b c', Domain)
solver1 = Solver()
solver1.add(Distinct(a, b, c)) ## Distinct !!
solver1.add(reflex, antisym, trans, restr_domain)
solver1.check()
solver1.model()

[b = Domain!val!1,
 a = Domain!val!0,
 c = Domain!val!2,
 P = [else ->
      Not(And(Var(0) == A!val!0, Not(Var(1) == A!val!0)))]]

###### Be aware of variable scope

In [94]:
x, y = Ints('x y')
solve([y == 1, ForAll([x,y], x-y==x-1)])

no solution


In [95]:
x, y, z = Ints('x y z')
solve([y == 1, ForAll([x,z], x-y==x-1)])

[y = 1]


#### Example: Sub-type relation

In [97]:
solver1 = Solver()
T = DeclareSort("Type") # (declare-sort Type)
subtype = Function("subtype",T,T, BoolSort()) # (declare-fun subtype (Type Type) Bool)
array_of = Function("array-of", T, T) #(declare-fun array-of (Type) Type)
root = Const("root", T) #(declare-const root-type Type)

# Reflexive
solver1.add(ForAll([x], subtype(x,x))) # (assert (forall ((x Type)) (subtype x x)))

# Transitive
solver1.add(ForAll([x,y,z], And(subtype(x,y), subtype(y,z)) == subtype(x,z)))
# (assert (forall ((x Type) (y Type) (z Type)) (= (and (subtype x y) (subtype y z)) (subtype x z)))) 

# Symmetric
solver1.add(ForAll([x,y], And(subtype(x,y), subtype(y,x)) == (x==y)))
#(assert (forall ((x Type) (y Type)) (= (and (subtype x y) (subtype y x)) (= x y))))

# Tree
solver1.add(ForAll([x,y,z], And(subtype(x,y), subtype(x,z)) == Or(subtype(y,z), subtype(z,y))))
#(assert (forall ((x Type) (y Type) (z Type)) 
#     (= (and (subtype x y) (subtype x z))  (or (subtype y z) (subtype z y))))) 

# Root
solver1.add(ForAll([x], subtype(x,root))) #(assert (forall ((x Type)) (subtype x root-type)))

# Array
solver1.add(ForAll([x,y], subtype(x,y)== subtype(array_of(x), array_of(y))))
#(assert (forall ((x Type) (y Type)) (= (subtype x y) (subtype (array-of x) (array-of y)))))


solver1.check() #(check-sat)
solver1.model()

Z3Exception: Sort mismatch

## Lambdas

In [98]:
a = Int('a') #(declare-const a Int)
b = Int('b') #(declare-const b Int)
c = Int('c') #(declare-const c Int)
L = Lambda([a,b,c], a+ b*c)
simplify(Select(L,a,b,c)) #(simplify (select (lambda ((x Int) (y Int) (z Int)) (+ x (* z y))) a b c))

a + b*c

In [99]:
simplify(Select(L,2,3,4)) #(simplify (select (lambda ((x Int) (y Int) (z Int)) (+ x (* z y))) 2 3 4))

14

## Recursion

In [100]:
fac = RecFunction('fac', IntSort(), IntSort())
n = Int('n')
m = Int('m')
RecAddDefinition(fac, n, If(n == 0, 1, n*fac(n-1)))
simplify(fac(5))

120

In [102]:
fac(m)

fac(m)

In [103]:
solver1 = Solver() 
solver1.add(fac(n) < 3)
solver1.check()
solver1.model()

[n = 0, fac = [0 -> 1, else -> Var(0)*fac(Var(0) - 1)]]

In [104]:
solver1 = Solver()
solver1.add(And(fac(n) < 3, fac(n) > 1))
solver1.check()
solver1.model()

[n = 2, fac = [0 -> 1, else -> Var(0)*fac(Var(0) - 1)]]

In [ ]:
#(declare-const n Int)
#(define-fun-rec fac ((n Int)) Int (if (= n 0) 1 (* n (fac (- n 1)))))
#(assert (< (fac n) 3))
#(check-sat)
#(get-model)

## Python specific things

In [105]:
x = Int('x')
y = Int('y')
n = x + y >= 3
print ("num args: ", n.num_args())
print ("children: ", n.children())
print ("1st child:", n.arg(0))
print ("2nd child:", n.arg(1))
print ("operator: ", n.decl())
print ("op name:  ", n.decl().name())

num args:  2
children:  [x + y, 3]
1st child: x + y
2nd child: 3
operator:  >=
op name:   >=


In [ ]:
x = Int('x')
y = Int('y')
n = And(x + y >= 3, x<2)
print ("num args: ", n.num_args())
print ("children: ", n.children())
print ("1st child:", n.arg(0))
print ("2nd child:", n.arg(1))
print ("operator: ", n.decl())
print ("op name:  ", n.decl().name())